In [15]:
"""Dataloading for Omniglot."""
import os
import glob

import google_drive_downloader as gdd
import imageio
import numpy as np
import torch
from torch.utils.data import dataset, sampler, dataloader

NUM_TRAIN_CLASSES = 1100
NUM_VAL_CLASSES = 100
NUM_TEST_CLASSES = 423
NUM_SAMPLES_PER_CLASS = 20


def load_image(file_path):
    """Loads and transforms an Omniglot image.

    Args:
        file_path (str): file path of image

    Returns:
        a Tensor containing image data
            shape (1, 28, 28)
    """
    x = imageio.imread(file_path)
    x = torch.tensor(x, dtype=torch.float32).reshape([1, 28, 28]) / x.max()
    return 1 - x


class OmniglotDataset(dataset.Dataset):
    """Omniglot dataset for meta-learning.

    Each element of the dataset is a task. A task is specified with a key,
    which is a tuple of class indices (no particular order). The corresponding
    value is the instantiated task, which consists of sampled (image, label)
    pairs.
    """
    _BASE_PATH = './omniglot_resized'
    if os.path.isdir('./submission'):
        _BASE_PATH = './submission/omniglot_resized'
    _GDD_FILE_ID = '1iaSFXIYC3AB8q9K_M-oVMa4pmB7yKMtI'

    def __init__(self, num_support, num_query):
        """Inits OmniglotDataset.

        Args:
            num_support (int): number of support examples per class
            num_query (int): number of query examples per class
        """
        super().__init__()


        # if necessary, download the Omniglot dataset
        if not os.path.isdir(self._BASE_PATH):
            gdd.GoogleDriveDownloader.download_file_from_google_drive(
                file_id=self._GDD_FILE_ID,
                dest_path=f'{self._BASE_PATH}.zip',
                unzip=True
            )

        # get all character folders
        self._character_folders = glob.glob(
            os.path.join(self._BASE_PATH, '*/*/'))
        assert len(self._character_folders) == (
            NUM_TRAIN_CLASSES + NUM_VAL_CLASSES + NUM_TEST_CLASSES
        )

        # shuffle characters
        np.random.default_rng(0).shuffle(self._character_folders)

        # check problem arguments
        assert num_support + num_query <= NUM_SAMPLES_PER_CLASS
        self._num_support = num_support
        self._num_query = num_query

    def __getitem__(self, class_idxs):
        """Constructs a task.

        Data for each class is sampled uniformly at random without replacement.

        Args:
            class_idxs (tuple[int]): class indices that comprise the task

        Returns:
            images_support (Tensor): task support images
                shape (num_way * num_support, channels, height, width)
            labels_support (Tensor): task support labels
                shape (num_way * num_support,)
            images_query (Tensor): task query images
                shape (num_way * num_query, channels, height, width)
            labels_query (Tensor): task query labels
                shape (num_way * num_query,)
        """
        images_support, images_query = [], []
        labels_support, labels_query = [], []

        for label, class_idx in enumerate(class_idxs):
            # get a class's examples and sample from them
            all_file_paths = glob.glob(
                os.path.join(self._character_folders[class_idx], '*.png')
            )
            sampled_file_paths = np.random.default_rng().choice(
                all_file_paths,
                size=self._num_support + self._num_query,
                replace=False
            )
            images = [load_image(file_path) for file_path in sampled_file_paths]

            # split sampled examples into support and query
            images_support.extend(images[:self._num_support])
            images_query.extend(images[self._num_support:])
            labels_support.extend([label] * self._num_support)
            labels_query.extend([label] * self._num_query)

        # aggregate into tensors
        images_support = torch.stack(images_support)  # shape (N*S, C, H, W)
        labels_support = torch.tensor(labels_support)  # shape (N*S)
        images_query = torch.stack(images_query)
        labels_query = torch.tensor(labels_query)

        return images_support, labels_support, images_query, labels_query


class OmniglotSampler(sampler.Sampler):
    """Samples task specification keys for an OmniglotDataset."""

    def __init__(self, split_idxs, num_way, num_tasks):
        """Inits OmniglotSampler.

        Args:
            split_idxs (range): indices that comprise the
                training/validation/test split
            num_way (int): number of classes per task
            num_tasks (int): number of tasks to sample
        """
        super().__init__(None)
        self._split_idxs = split_idxs
        self._num_way = num_way
        self._num_tasks = num_tasks

    def __iter__(self):
        return (
            np.random.default_rng().choice(
                self._split_idxs,
                size=self._num_way,
                replace=False
            ) for _ in range(self._num_tasks)
        )

    def __len__(self):
        return self._num_tasks


def identity(x):
    return x


def get_omniglot_dataloader(
        split,
        batch_size,
        num_way,
        num_support,
        num_query,
        num_tasks_per_epoch,
        num_workers=2,
):
    """Returns a dataloader.DataLoader for Omniglot.

    Args:
        split (str): one of 'train', 'val', 'test'
        batch_size (int): number of tasks per batch
        num_way (int): number of classes per task
        num_support (int): number of support examples per class
        num_query (int): number of query examples per class
        num_tasks_per_epoch (int): number of tasks before DataLoader is
            exhausted
    """

    if split == 'train':
        split_idxs = range(NUM_TRAIN_CLASSES)
    elif split == 'val':
        split_idxs = range(
            NUM_TRAIN_CLASSES,
            NUM_TRAIN_CLASSES + NUM_VAL_CLASSES
        )
    elif split == 'test':
        split_idxs = range(
            NUM_TRAIN_CLASSES + NUM_VAL_CLASSES,
            NUM_TRAIN_CLASSES + NUM_VAL_CLASSES + NUM_TEST_CLASSES
        )
    else:
        raise ValueError

    return dataloader.DataLoader(
        dataset=OmniglotDataset(num_support, num_query),
        batch_size=batch_size,
        sampler=OmniglotSampler(split_idxs, num_way, num_tasks_per_epoch),
        num_workers=num_workers,
        collate_fn=identity,
        pin_memory=torch.cuda.is_available(),
        drop_last=True
    )


In [7]:
train_loader = get_omniglot_dataloader('train', 10, 5, 5, 10, 100)

Unzipping...Done.


In [8]:
network = ProtoNetNetwork("cpu")

In [ ]:
for batch in train_loader:
  for task in batch:
    images_support, labels_support, images_query, labels_query = task

    supp_feats = network(images_support)

    classes = torch.unique(labels_support, sorted=True)
    prototypes = []
    for c in classes:
        idxes = (labels_support == c).nonzero()
        class_feats = supp_feats[idxes]
        prototype = torch.mean(class_feats, dim=0)
        prototypes.append(prototype)

    prototypes = torch.cat(prototypes)  # (num_classes, feature_dim)

    query_feats = network(images_query)

    print(query_feats.shape)

    query_feats = torch.stack([query_feats] * prototypes.size(0), dim=1)
    print(query_feats.shape)

    exp_means = prototypes.expand(*query_feats.shape)

    print(exp_means.shape)

    break

  break

In [10]:
means.shape

NameError: ignored

In [16]:
"""Utilities for scoring the model."""
import torch


def score(logits, labels):
    """Returns the mean accuracy of a model's predictions on a set of examples.

    Args:
        logits (torch.Tensor): model predicted logits
            shape (examples, classes)
        labels (torch.Tensor): classification labels from 0 to num_classes - 1
            shape (examples,)
    """

    assert logits.dim() == 2
    assert labels.dim() == 1
    assert logits.shape[0] == labels.shape[0]
    y = torch.argmax(logits, dim=-1) == labels
    y = y.type(torch.float)
    return torch.mean(y).item()

In [17]:
"""Implementation of prototypical networks for Omniglot."""
import sys
sys.path.append('..')
import argparse
import os

import numpy as np
import torch

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

from torch import nn
import torch.nn.functional as F  # pylint: disable=unused-import
from google_drive_downloader import GoogleDriveDownloader as gdd
from torch.utils import tensorboard

NUM_INPUT_CHANNELS = 1
NUM_HIDDEN_CHANNELS = 64
KERNEL_SIZE = 3
NUM_CONV_LAYERS = 4
SUMMARY_INTERVAL = 10
SAVE_INTERVAL = 100
PRINT_INTERVAL = 10
VAL_INTERVAL = PRINT_INTERVAL * 5
NUM_TEST_TASKS = 600

class ProtoNetNetwork(nn.Module):
    """Container for ProtoNet weights and image-to-latent computation."""

    def __init__(self, device):
        """Inits ProtoNetNetwork.

        The network consists of four convolutional blocks, each comprising a
        convolution layer, a batch normalization layer, ReLU activation, and 2x2
        max pooling for downsampling. There is an additional flattening
        operation at the end.

        Note that unlike conventional use, batch normalization is always done
        with batch statistics, regardless of whether we are training or
        evaluating. This technically makes meta-learning transductive, as
        opposed to inductive.

        Args:
            device (str): device to be used
        """
        super().__init__()
        layers = []
        in_channels = NUM_INPUT_CHANNELS
        for _ in range(NUM_CONV_LAYERS):
            layers.append(
                nn.Conv2d(
                    in_channels,
                    NUM_HIDDEN_CHANNELS,
                    (KERNEL_SIZE, KERNEL_SIZE),
                    padding='same'
                )
            )
            layers.append(nn.BatchNorm2d(NUM_HIDDEN_CHANNELS))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2))
            in_channels = NUM_HIDDEN_CHANNELS
        layers.append(nn.Flatten())
        self._layers = nn.Sequential(*layers)
        self.to(device)

    def forward(self, images):
        """Computes the latent representation of a batch of images.

        Args:
            images (Tensor): batch of Omniglot images
                shape (num_images, channels, height, width)

        Returns:
            a Tensor containing a batch of latent representations
                shape (num_images, latents)
        """
        return self._layers(images)


class ProtoNet:
    """Trains and assesses a prototypical network."""

    def __init__(self, learning_rate, log_dir, device, compile=False, backend=None, learner=None, val_interval=None, save_interval=None, bio=False):
        """Inits ProtoNet.

        Args:
            learning_rate (float): learning rate for the Adam optimizer
            log_dir (str): path to logging directory
            device (str): device to be used
        """
        self.device = device
        if learner is None:
            self._network = ProtoNetNetwork(device)
        else:
            self._network = learner.to(device)

        self.val_interval = VAL_INTERVAL if val_interval is None else val_interval
        self.save_interval = SAVE_INTERVAL if save_interval is None else save_interval
        self.bio = bio

        if(compile == True):
            try:
                self._network = torch.compile(self._network, backend=backend)
                print(f"ProtoNetNetwork model compiled")
            except Exception as err:
                print(f"Model compile not supported: {err}")

        self._optimizer = torch.optim.Adam(
            self._network.parameters(),
            lr=learning_rate
        )
        self._log_dir = log_dir
        os.makedirs(self._log_dir, exist_ok=True)

        self._start_train_step = 0

    def _step(self, task_batch):
        """Computes ProtoNet mean loss (and accuracy) on a batch of tasks.

        Args:
            task_batch (tuple[Tensor, Tensor, Tensor, Tensor]):
                batch of tasks from an Omniglot DataLoader

        Returns:
            a Tensor containing mean ProtoNet loss over the batch
                shape ()
            mean support set accuracy over the batch as a float
            mean query set accuracy over the batch as a float
        """
        loss_batch = []
        accuracy_support_batch = []
        accuracy_query_batch = []
        for i, task in enumerate(task_batch):
            # print(i)
            images_support, labels_support, images_query, labels_query = task
            # print(images_support.shape, labels_support.shape, images_query.shape, labels_query.shape)
            images_support = images_support.to(self.device)
            labels_support = labels_support.to(self.device)
            images_query = images_query.to(self.device)
            labels_query = labels_query.to(self.device)

            ### START CODE HERE ###
            # TODO: finish implementing this method.
            # For a given task, compute the prototypes and the protonet loss.
            # Use F.cross_entropy to compute classification losses.
            # Use util.score to compute accuracies.
            # Make sure to populate loss_batch, accuracy_support_batch, and
            # accuracy_query_batch.

            # compute prototypes without tracking gradients
                # Generate support features
            supp_feats = self._network(images_support)

            classes = torch.unique(labels_support, sorted=True)
            prototypes = []
            for c in classes:
                idxes = (labels_support == c).nonzero()
                class_feats = supp_feats[idxes]
                prototype = torch.mean(class_feats, dim=0)
                prototypes.append(prototype)

            prototypes = torch.cat(prototypes)  # (num_classes, feature_dim)

            # now to measure distances to all means
            # (batch_size, feature_dim)
            query_feats = self._network(images_query)
            # (batch_size, num_classes, feature_dim)
            query_feats = torch.stack([query_feats] * prototypes.size(0), dim=1)

            expanded_prototypes = prototypes.expand(*query_feats.shape)

            # (batch_size, num_classes, feature_dim)
            query_diffs = query_feats - expanded_prototypes
            # (batch_size, num_classes)
            query_sq_norms = torch.norm(query_diffs, p=2, dim=-1).square()
            query_logits = -query_sq_norms

            loss = F.cross_entropy(query_logits, labels_query)
            loss_batch.append(loss)

            # compute accuracies
            query_acc = score(query_logits, labels_query)
            accuracy_query_batch.append(query_acc)

            # for calculating acc and loss for support set

            # (batch_size, num_classes, feature_dim)
            supp_feats = torch.stack([supp_feats] * prototypes.size(0), dim=1)

            expanded_prototypes = prototypes.expand(*supp_feats.shape)

            # (batch_size, num_classes, feature_dim)
            supp_diffs = supp_feats - expanded_prototypes
            # (batch_size, num_classes)
            supp_sq_norms = torch.norm(supp_diffs, p=2, dim=2).square()
            supp_logits = -supp_sq_norms
            supp_acc = score(supp_logits, labels_support)
            accuracy_support_batch.append(supp_acc)

            ### END CODE HERE ###
        return (
            torch.mean(torch.stack(loss_batch)),
            np.mean(accuracy_support_batch),
            np.mean(accuracy_query_batch)
        )

    def train(self, dataloader_meta_train, dataloader_meta_val, writer):
        """Train the ProtoNet.

        Consumes dataloader_meta_train to optimize weights of ProtoNetNetwork
        while periodically validating on dataloader_meta_val, logging metrics, and
        saving checkpoints.

        Args:
            dataloader_meta_train (DataLoader): loader for train tasks
            dataloader_meta_val (DataLoader): loader for validation tasks
            writer (SummaryWriter): TensorBoard logger
        """
        print(f'Starting training at iteration {self._start_train_step}.')
        MAX_TRAIN = len(dataloader_meta_train)
        # exit()
        for i_step, task_batch in enumerate(
                dataloader_meta_train,
                start=self._start_train_step
        ):
            if i_step > MAX_TRAIN:
                break
            self._optimizer.zero_grad()
            loss, accuracy_support, accuracy_query = self._step(task_batch)
            loss.backward()
            self._optimizer.step()

            if i_step % PRINT_INTERVAL == 0:
                print(
                    f'Iteration {i_step}: '
                    f'loss: {loss.item():.3f}, '
                    f'support accuracy: {accuracy_support.item():.3f}, '
                    f'query accuracy: {accuracy_query.item():.3f}'
                )
                writer.add_scalar('loss/train', loss.item(), i_step)
                writer.add_scalar(
                    'train_accuracy/support',
                    accuracy_support.item(),
                    i_step
                )
                writer.add_scalar(
                    'train_accuracy/query',
                    accuracy_query.item(),
                    i_step
                )

            if i_step % self.val_interval == 0:
                print("Start Validation...")
                with torch.no_grad():
                    losses, accuracies_support, accuracies_query = [], [], []
                    for i, val_task_batch in enumerate(dataloader_meta_val):
                        if self.bio and i > 600:
                            break
                        loss, accuracy_support, accuracy_query = (
                            self._step(val_task_batch)
                        )
                        losses.append(loss.item())
                        accuracies_support.append(accuracy_support)
                        accuracies_query.append(accuracy_query)
                    loss = np.mean(losses)
                    accuracy_support = np.mean(accuracies_support)
                    accuracy_query = np.mean(accuracies_query)
                    ci95 = 1.96 * np.std(accuracies_query) / np.sqrt(600 * 4)
                if self.bio:
                    print(
                        f'Validation: '
                        f'loss: {loss:.3f}, '
                        f'support accuracy: {accuracy_support:.3f}, '
                        f'query accuracy: {accuracy_query:.3f}',
                        f'Ci95: {ci95:.3f}'
                    )
                else:
                    print(
                        f'Validation: '
                        f'loss: {loss:.3f}, '
                        f'support accuracy: {accuracy_support:.3f}, '
                        f'query accuracy: {accuracy_query:.3f}'
                    )
                writer.add_scalar('loss/val', loss, i_step)
                writer.add_scalar(
                    'val_accuracy/support',
                    accuracy_support,
                    i_step
                )
                writer.add_scalar(
                    'val_accuracy/query',
                    accuracy_query,
                    i_step
                )
                if self.bio:
                    writer.add_scalar(
                        'val_accuracy/ci95',
                        ci95,
                        i_step
                    )
            if i_step % self.save_interval == 0:
                self._save(i_step)

    def test(self, dataloader_test):
        """Evaluate the ProtoNet on test tasks.

        Args:
            dataloader_test (DataLoader): loader for test tasks
        """
        accuracies = []
        for i, task_batch in enumerate(dataloader_test):
            accuracies.append(self._step(task_batch)[2])
        mean = np.mean(accuracies)
        std = np.std(accuracies)
        mean_95_confidence_interval = 1.96 * std / np.sqrt(NUM_TEST_TASKS)
        print(
            f'Accuracy over {NUM_TEST_TASKS} test tasks: '
            f'mean {mean:.3f}, '
            f'95% confidence interval {mean_95_confidence_interval:.3f}'
        )

    def load(self, checkpoint_step, filename=""):
        """Loads a checkpoint.

        Args:
            checkpoint_step (int): iteration of checkpoint to load
            filename (str): directly setting name of checkpoint file, default ="", when argument is passed, then checkpoint will be ignored

        Raises:
            ValueError: if checkpoint for checkpoint_step is not found
        """
        target_path = (
            f'{os.path.join(self._log_dir, "state")}'
            f'{checkpoint_step}.pt'
        ) if filename == "" else filename
        if os.path.isfile(target_path):
            state = torch.load(target_path)
            self._network.load_state_dict(state['network_state_dict'])
            self._optimizer.load_state_dict(state['optimizer_state_dict'])
            self._start_train_step = checkpoint_step + 1
            print(f'Loaded checkpoint iteration {checkpoint_step}.')
        else:
            raise ValueError(
                f'No checkpoint for iteration {checkpoint_step} found.'
            )

    def _save(self, checkpoint_step):
        """Saves network and optimizer state_dicts as a checkpoint.

        Args:
            checkpoint_step (int): iteration to label checkpoint with
        """
        torch.save(
            dict(network_state_dict=self._network.state_dict(),
                 optimizer_state_dict=self._optimizer.state_dict()),
            f'{os.path.join(self._log_dir, "state")}{checkpoint_step}.pt'
        )
        print('Saved checkpoint.')


def main(args):

    print(args)

    if args.device == "gpu" and torch.backends.mps.is_available() and torch.backends.mps.is_built():
        DEVICE = "mps"
    elif args.device == "gpu" and torch.cuda.is_available():
        DEVICE = "cuda"
    else:
        DEVICE = "cpu"

    print("Using device: ", DEVICE)

    log_dir = args.log_dir
    if log_dir is None:
        log_dir = f'./logs/protonet/omniglot.way_{args.num_way}.support_{args.num_support}.query_{args.num_query}.lr_{args.learning_rate}.batch_size_{args.batch_size}'  # pylint: disable=line-too-long
    print(f'log_dir: {log_dir}')
    writer = tensorboard.SummaryWriter(log_dir=log_dir)

    protonet = ProtoNet(args.learning_rate, log_dir, DEVICE, args.compile, args.backend)

    if args.checkpoint_step > -1:
        protonet.load(args.checkpoint_step)
    else:
        print('Checkpoint loading skipped.')

    if not args.test:
        num_training_tasks = args.batch_size * (args.num_train_iterations -
                                                args.checkpoint_step - 1)
        print(
            f'Training on tasks with composition '
            f'num_way={args.num_way}, '
            f'num_support={args.num_support}, '
            f'num_query={args.num_query}'
        )
        dataloader_meta_train = get_omniglot_dataloader(
            'train',
            args.batch_size,
            args.num_way,
            args.num_support,
            args.num_query,
            num_training_tasks,
            args.num_workers
        )
        dataloader_meta_val = get_omniglot_dataloader(
            'val',
            args.batch_size,
            args.num_way,
            args.num_support,
            args.num_query,
            args.batch_size * 4,
            args.num_workers,
        )
        protonet.train(
            dataloader_meta_train,
            dataloader_meta_val,
            writer
        )
    else:
        print(
            f'Testing on tasks with composition '
            f'num_way={args.num_way}, '
            f'num_support={args.num_support}, '
            f'num_query={args.num_query}'
        )
        dataloader_test =  get_omniglot_dataloader(
            'test',
            1,
            args.num_way,
            args.num_support,
            args.num_query,
            NUM_TEST_TASKS,
            args.num_workers
        )
        protonet.test(dataloader_test)




In [18]:
class Args:
  def __init__(self):
    self.log_dir = None
    self.num_way = 5
    self.num_support = 5
    self.num_query = 5
    self.learning_rate = 0.001
    self.batch_size = 16
    self.num_train_iterations = 2000
    self.test = False
    self.checkpoint_step = -1
    self.num_workers = 2
    self.device = "gpu"
    self.compile = False
    self.cache = False
    self.backend = "inductor"



In [14]:
args = Args()
main(args)

Using device:  cuda
log_dir: ./logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16
Checkpoint loading skipped.
Training on tasks with composition num_way=5, num_support=5, num_query=5
Starting training at iteration 0.


<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)
<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)


Iteration 0: loss: 0.849, support accuracy: 0.992, query accuracy: 0.767
Start Validation...


<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)
<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)


Validation: loss: 0.537, support accuracy: 0.987, query accuracy: 0.837
Saved checkpoint.
Iteration 10: loss: 0.269, support accuracy: 0.980, query accuracy: 0.902
Iteration 20: loss: 0.329, support accuracy: 0.987, query accuracy: 0.922
Iteration 30: loss: 0.099, support accuracy: 0.997, query accuracy: 0.965
Iteration 40: loss: 0.097, support accuracy: 1.000, query accuracy: 0.967
Iteration 50: loss: 0.105, support accuracy: 0.997, query accuracy: 0.965
Start Validation...


...

Validation: loss: 0.017, support accuracy: 0.999, query accuracy: 0.994
Iteration 4860: loss: 0.025, support accuracy: 1.000, query accuracy: 0.997
Iteration 4870: loss: 0.005, support accuracy: 0.997, query accuracy: 0.997
Iteration 4880: loss: 0.009, support accuracy: 1.000, query accuracy: 0.995
Iteration 4890: loss: 0.001, support accuracy: 1.000, query accuracy: 1.000
Iteration 4900: loss: 0.005, support accuracy: 1.000, query accuracy: 0.997
Start Validation...


<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)
<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)


Validation: loss: 0.011, support accuracy: 0.999, query accuracy: 0.995
Saved checkpoint.
Iteration 4910: loss: 0.007, support accuracy: 1.000, query accuracy: 0.997
Iteration 4920: loss: 0.004, support accuracy: 1.000, query accuracy: 0.997
Iteration 4930: loss: 0.004, support accuracy: 1.000, query accuracy: 1.000
Iteration 4940: loss: 0.007, support accuracy: 1.000, query accuracy: 0.997
Iteration 4950: loss: 0.001, support accuracy: 1.000, query accuracy: 1.000
Start Validation...


<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)
<ipython-input-6-079f4bf87d5c>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imageio.imread(file_path)


Validation: loss: 0.021, support accuracy: 1.000, query accuracy: 0.995
Iteration 4960: loss: 0.008, support accuracy: 1.000, query accuracy: 0.997
Iteration 4970: loss: 0.009, support accuracy: 1.000, query accuracy: 0.995
Iteration 4980: loss: 0.007, support accuracy: 1.000, query accuracy: 0.997
Iteration 4990: loss: 0.005, support accuracy: 1.000, query accuracy: 1.000


In [30]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [33]:
%tensorboard --logdir=/content/logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16 --port=4005

<IPython.core.display.Javascript object>

In [21]:
!zip -r prototypical_network_logs.zip logs/

  adding: logs/ (stored 0%)
  adding: logs/protonet/ (stored 0%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/ (stored 0%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/state3300.pt (deflated 8%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/state2600.pt (deflated 8%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/state2700.pt (deflated 8%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/state500.pt (deflated 8%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/state1900.pt (deflated 8%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/state2300.pt (deflated 8%)
  adding: logs/protonet/omniglot.way_5.support_5.query_5.lr_0.001.batch_size_16/events.out.tfevents.1699966391.9f080bafb459.1333.1 (deflated 74%)
  adding: logs/protonet/omniglot.way_5.support_5.query_